In [2]:
import numpy as np
from numpy import log, exp, zeros, dot, array
import itertools
from itertools import product
import timeit

In [3]:
def log_sum_exp(x):
    
    m = x.max()
    x_s = x - m
    return m + log((exp(x_s)).sum())


def label_seq(l,j):
    '''Creates a list of all posible length l sequences taking values in
       {0,1,...,j-1}. Out put list is of length j^l'''
    
    
    return list(product(list(range(j)), repeat=l))

def find_legit_vals(k,l,n):
    '''Finds the only possible label pairs (i,j) with f_k(i,j,x,t) != 0.
       Will be used to speed up gradient computation.
       l: Number of labels
       n: Size of a nodes feature vector'''
    
    res = []
    k_s = k - n*l
    
    if k_s < 0:  #Means k is in the Unary features
        
        i = k // n
        
        for j in range(l):
            
            res.append((i,j))
            
    else:  #Means k is a Binary feature
        
        f = (k_s // n)
        j = f % l
        i = f // l
        res.append((i,j))  
    
    return res


class CRF(object):
    
    def __init__(self, feature_function, K, T, L, lamb, W='n'):
        ''' If our labels belong to space L and our observations belonging to X amd
            then length of our chain is T (call {0,1, ... , T-1} = T') then 
            feature_functions is a vector valued function, f: L^2 x X x T' --> R^K
            i.e. f(i,j,x,t) is a K-d real valued vector and has component functions of the form
            f_k(i,j,x,t) to be specified, think of (i,j) = (y_t, y_t-1). It will have an optional
            keyword argument 'project' which defaults to -1, if a positive integer k,
            it will project onto the k-th component.
            
            K: # of Features
            
            T: Length of Chain
            
            L: Number of labels. Will assume labels have been encoded as integers in {0,...,L-1} 
            
            lamb: L2 regularization constant'''
        
        self.f_x = feature_function
        if type(W) == str:
            self.W = np.random.uniform(low=-0.5, high=0.5, size=K)
        else:
            self.W = W
        self.K = K
        self.T = T
        self.L = L
        self.Lambda = lamb
        
    def  log_forward(self, x):
        '''This computes the log(alphas) as in the forward-backward algorithm in order to
           be used for inference tasks later on.
           x is an observation.'''
        
        f = self.f_x
        alphas = zeros((self.T, self.L))
        
        # Initialization
        
        for l in range(self.L):
            
            alphas[0,l] = dot(self.W, f(l,0,x,0))
            
        # Recursion
        
        for t in range(1,self.T):
            
            for l in range(self.L):
                
                psi = array([dot(self.W, f(l,i,x,t)) for i in range(self.L)])
                
                alphas[t,l] = log_sum_exp(psi + alphas[t-1])
            
        return alphas
    
        
    def log_backward(self, x):
        '''This computes the log(betas) as in the forward-backward algorithm in order to
           be used for inference tasks later on.
           x is an observation.'''
        
        # Initialization
        
        f = self.f_x
        betas = np.ones((self.T, self.L))
        
        # Recursion
        
        for t in range(self.T-2,-1,-1):
            
            for l in range(self.L):
                
                psi = array([dot(self.W, f(i,l,x,t+1)) for i in range(self.L)])
                
                betas[t][l] = log_sum_exp(psi + betas[t+1])
                
        return betas
    
    
    def log_partition(self, x):
        '''Efficient computation of the log of the partition function Z(x) appearing in CRF model.
           Input an observation and inital label (for forward algorithm) and output is log(Z(x))'''
        
        alphas = self.log_forward(x)
        
        return log_sum_exp(alphas[-1])
    
    
    def MAP(self, x):
        '''Viterbi algortithm for computing the most likely label of a sequence with
           given observation vector x using maximum a posteriori estimation. Using log
           sum version for numeric stability'''
        
        f = self.f_x
            
        # Initialization
        
        deltas = np.zeros((self.T, self.L))
        delt_arg = np.zeros((self.T, self.L))
        
        for l in range(self.L):
            
            deltas[0][l] = dot(self.W, f(l,0,x,0))  # Not sure about this.
            
        # Recursion
        
        for t in range(1,self.T):
            
            for l in range(self.L):
                
                psi = array([dot(self.W, f(l,i,x,t)) for i in range(self.L)])
                
                deltas[t][l] = (psi + deltas[t-1]).max()
                delt_arg[t][l] = (psi + deltas[t-1]).argmax()
            
        map_lab = np.zeros(self.T, dtype='i4')
        map_lab[-1] = deltas[-1].argmax()
        
        for t in range(self.T-2,-1,-1):
            
            map_lab[t] = delt_arg[t+1][map_lab[t+1]]
            
                
        return tuple(map_lab)
        
        
    def marginal(self,i,j,x,t):
        '''Using the forward backward algorithm to compute the marginal p(y_t-1=i,y_t=j|x)'''
        
        f = self.f_x
        alphas = self.log_forward(x)
        betas = self.log_backward(x)
        psi = dot(self.W,f(j,i,x,t))
        psi_b = np.array([dot(self.W,f(k,0,x,0)) for k in range(self.L)])
        log_joint = alphas[t-1][i] + psi + betas[t][j] - log_sum_exp(psi_b + betas[0])
        
        return exp(log_joint)
                       
    def naive_comp(self, x, out='Z'):
        '''Brute force computation of log(Z(x)) or MAP (if out = 'MAP')'''
        
        f = self.f_x
        
        # Get List of all possible label sequences
        
        lab_seq = label_seq(self.T, self.L)
        
        psi = np.zeros(len(lab_seq))
        
        for k in range(len(lab_seq)):
            
            lab = lab_seq[k]
            temp = np.zeros(self.T)
            temp[0] = dot(self.W, f(lab[0], 0, x, 0))
            
            for t in range(1,self.T):
                
                temp[t] = dot(self.W, f(lab[t], lab[t-1], x, t))
            
            psi[k] = temp.sum()
        
        arg_m_i = psi.argmax()
        
        return log_sum_exp(psi) if out == 'Z' else lab_seq[arg_m_i]
        
        
    
    def gradient(self, X, Y):
        ''' Creates the gradient vector of the log-likelihood. 
            X, Y: Are arrays containing training examples.'''
        
        f = self.f_x
        lamb = self.Lambda
        grad = np.zeros(self.K)
        
        for k in range(self.K):
            
            val_pair = find_legit_vals(k, self.L, X.shape[-1])
            first_term = np.zeros((X.shape[0],self.T))
            
            for n in range(X.shape[0]):
                
                for t in range(self.T):
                    
                    first_term[n][t] = f(Y[n][t], Y[n][t-1], X[n], t, project=k)
            
            sec_term = np.zeros((X.shape[0],self.T))
            
            for n in range(X.shape[0]):
                
                for t in range(self.T):
                    
                    marginals = np.zeros(len(val_pair))
                        
                    for j in range(len(val_pair)):
                        
                        y, y_p = val_pair[j]
                        marginals[j] = (f(y, y_p, X[n], t, project=k) * 
                                        self.marginal(y, y_p, X[n], t))
                    
                    sec_term[n][t] = marginals.sum()
                    
            grad[k] = (first_term + sec_term).sum() - self.W[k] * lamb
            print(k)
        return grad
    
    
    def gradient_f(self, X, Y):
        
        f = self.f_x
        lamb = self.Lambda
        grad_f = np.zeros(self.K)
        lab_pairs = label_seq(2,self.L)
        
        for n in range(X.shape[0]):
            
            grad = np.zeros(self.K)
            x, y = X[n], Y[n]
            
            for t in range(self.T):
            
                grad = grad + f(y[t],y[t-1],x,t)
        
            for pair in lab_pairs:
            
                y, y_p = pair[0], pair[1]
                grad_temp = np.zeros(self.K)
            
                for t in range(self.T):
                
                    marg = self.marginal(y,y_p,x,t)
                    grad_temp = grad_temp + (marg*f(y,y_p,x,t))
        
                grad = grad - grad_temp
            
            grad_f = grad_f + grad
    
        return lamb * self.W  - grad_f
    
    def reg_neg_ll(self, X, Y):
        
        f = self.f_x
        lamb = self.Lambda
        res = 0
        for n in range(X.shape[0]):
            
            s = 0
            
            for t in range(self.T):
            
                s += np.dot(self.W,f(Y[n,t],Y[n,t-1],X[n],t))
            
            res += self.log_partition(X[n]) - s
         
        return res + (0.5 * lamb * np.dot(self.W, self.W))
        

In [5]:
def feat_map(i, j, x, t, projection=-1):
    
    unary_chi = np.zeros(15)
    binary_chi = np.zeros(45)
    
    unary_chi[5*i:5*(i+1)] = (t >=0) * x[t]
    
    if t > 0:
        
        binary_chi[5*(3*i+j): 5*(3*i+j+1)] = x[t] + x[(t-1)]
        
    if t == 0 and j == 0:
        
        binary_chi[5*(3*i+j): 5*(3*i+j+1)] = x[0]
    
    output = np.zeros(3*5 + 9*5)
    output[:3*5] = unary_chi
    output[3*5:] = binary_chi
    
    if projection > 0:
        
        return output[projection]
    
    else:
        
        return binary_chi

In [37]:
# Generate simple test data: Length 5 chains, 3 class labels, each feature vector is 5 dimensional

features = np.zeros((99, 5, 5))

for k in range(33):
    
    for j in range(5):
        
        features[k,j,0] = np.random.randn()
        features[k+33,j,2] = np.random.randn()
        features[k+66,j,4] = np.random.randn()

labels = np.zeros(99)

for k in range(33):
    
    labels[k+33] = 1
    labels[k+66] = 2


In [38]:
features[0]

array([[-0.31992262,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.05266409,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.09258435,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 2.1222034 ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.43536307,  0.        ,  0.        ,  0.        ,  0.        ]])

In [124]:
crf_test = CRF(feat_map, 45, 5, 3, 1)

In [125]:
crf_test.MAP(features[1])

(0, 0, 1, 2, 2)

In [126]:
crf_test.naive_comp(features[1],'MAP')

(0, 0, 1, 2, 2)

In [127]:
crf_test.naive_comp(features[0])

7.5213871355553366

In [128]:
crf_test.log_partition(features[0])

7.5213871355553374

In [129]:
m_test = np.array([crf_test.marginal(i,j,features[12],2) for i in range(3) for j in range(3)])

In [130]:
m_test.sum()

0.99999999999999978

In [6]:
from oct2py import octave
octave.addpath('/home/james/anaconda3/data/HW')

'.:/home/james/anaconda3/lib/python3.5/site-packages/oct2py:/home/james/anaconda3/lib/python3.5/site-packages/octave_kernel:/usr/lib/x86_64-linux-gnu/octave/4.0.3/site/oct/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/api-v50+/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/x86_64-pc-linux-gnu:/usr/share/octave/4.0.3/site/m:/usr/share/octave/site/api-v50+/m:/usr/share/octave/site/m:/usr/share/octave/site/m/startup:/usr/lib/x86_64-linux-gnu/octave/4.0.3/oct/x86_64-pc-linux-gnu:/usr/share/octave/4.0.3/m:/usr/share/octave/4.0.3/m/audio:/usr/share/octave/4.0.3/m/debian:/usr/share/octave/4.0.3/m/deprecated:/usr/share/octave/4.0.3/m/elfun:/usr/share/octave/4.0.3/m/general:/usr/share/octave/4.0.3/m/geometry:/usr/share/octave/4.0.3/m/gui:/usr/share/octave/4.0.3/m/help:/usr/share/octave/4.0.3/m/image:/usr/share/octave/4.0.3/m/io:/usr/share/octave/4.0.3/m/java:/usr/share/octave/4.0.3/m/linear-algebra:/usr/share/octave/4.0.3/m/miscellaneous:/usr/share/octave/4.0.3/m/

In [22]:
X = octave.data_generator()

In [52]:
def feat_map_toy(i, j, x, t, project=-1):
    ''' Change the values of feature vector length and number of classes. '''
    
    N = 3                            # length of data vector
    L = 3                          # number of labels
    T = 10                          # length of sequence
    unary_chi = np.zeros(N * L)
    binary_chi = np.zeros(N * L**2)
    
    unary_chi[N*i:N*(i+1)] = x[t]
    
    if t > 0:
        
        binary_chi[N*(L*i+j): N*(L*i+j+1)] = x[t] + x[(t-1)]
        
    if t == 0 and j == 0:
        
        binary_chi[N*(L*i+j): N*(L*i+j+1)] = x[t]
    
    
    output = np.zeros(N*L + N*L**2)
    output[:N*L] = unary_chi
    output[N*L:] = binary_chi
    
    if project > -1:
        
        return output[project]
    
    else:
        
        return output

In [24]:
X_f = np.zeros((1000,10,3))

for k in range(1000):
    
    for j in range(10):
        
        X_f[k,j,:] = X[k][0][:,j]

In [25]:
y_f = np.zeros((1000,10))

for k in range(1000):
    
    y_f[k] = X[k,1][0] -1
    
y_f = np.asarray(y_f,dtype='i4')

In [26]:
y_f

array([[2, 2, 2, ..., 2, 2, 2],
       [1, 1, 1, ..., 0, 0, 0],
       [2, 2, 2, ..., 2, 2, 2],
       ..., 
       [2, 2, 2, ..., 2, 2, 2],
       [1, 1, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 2, 2, 2]], dtype=int32)

In [54]:
# Train and Test Split

dims = np.arange(X_f.shape[0])
np.random.shuffle(dims)

X_tr, y_tr = X_f[dims[:-100]], y_f[dims[:-100]]
X_ts, y_ts = X_f[dims[-100:]], y_f[dims[-100:]]

In [61]:
# Adam stochastic mini-batch gradient descent on the (negative) log liklelihood.
from scipy.spatial.distance import hamming 

crf_test1 = CRF(feat_map_toy, 3**2+ 3**3,10, 3, 0)
alpha = 0.03
B1, B2 = 0.9, 0.999
eps = 1e-7
m, v = np.zeros(crf_test1.K), np.zeros(crf_test1.K)
num_epochs = 3
batch_size = 16
epoch = 1

while epoch <= num_epochs:
    
    min_batch = np.arange(X_tr.shape[0])
    np.random.shuffle(min_batch)
    
    for k in range(0, X_tr.shape[0],batch_size):
        
        X_b, y_b = X_tr[min_batch[k:k+batch_size]],y_tr[min_batch[k:k+batch_size]]
        t = (X_tr.shape[0]//batch_size)*(epoch-1) + (k // 4) + 1
        g = crf_test1.gradient_f(X_b, y_b)
        m = B1 * m + (1-B1) * g
        v = B2 * v + np.square(np.sqrt(1-B2) * g)
        m_b = m / (1 - B1**t)
        v_b = v / (1 - B2**t)
        crf_test1.W = crf_test1.W - (alpha * m_b) / (np.sqrt(v_b) + eps)
        
        if (k // batch_size) % 5 == 0:
            
            acc_test = np.zeros(30)
            
            for n in range(30):
                acc_test[n] = 1 - hamming(crf_test1.MAP(X_ts[n]), y_ts[n])
            print('Epoch/Iteration: ', epoch, '/', t-1, '. Current Average Test Hamming Accuracy: ',
                  100* acc_test.mean(), '%.')
        
        if acc_test.mean() >= 0.96:
            break
            epoch = num_epochs+1
        
          
    epoch += 1
            

Epoch/Iteration:  1 / 0 . Current Average Test Hamming Accuracy:  35.6666666667 %.
Epoch/Iteration:  1 / 20 . Current Average Test Hamming Accuracy:  40.0 %.
Epoch/Iteration:  1 / 40 . Current Average Test Hamming Accuracy:  59.0 %.
Epoch/Iteration:  1 / 60 . Current Average Test Hamming Accuracy:  74.0 %.
Epoch/Iteration:  1 / 80 . Current Average Test Hamming Accuracy:  77.3333333333 %.
Epoch/Iteration:  1 / 100 . Current Average Test Hamming Accuracy:  79.0 %.
Epoch/Iteration:  1 / 120 . Current Average Test Hamming Accuracy:  82.3333333333 %.
Epoch/Iteration:  1 / 140 . Current Average Test Hamming Accuracy:  86.6666666667 %.
Epoch/Iteration:  1 / 160 . Current Average Test Hamming Accuracy:  93.0 %.
Epoch/Iteration:  1 / 180 . Current Average Test Hamming Accuracy:  94.6666666667 %.
Epoch/Iteration:  1 / 200 . Current Average Test Hamming Accuracy:  96.0 %.
Epoch/Iteration:  2 / 56 . Current Average Test Hamming Accuracy:  96.0 %.
Epoch/Iteration:  3 / 112 . Current Average Test H

In [31]:
crf_test1.MAP(X_f[8])

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

In [32]:
y_f[8]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [63]:
results = np.zeros(100)
for k in range(100):
    results[k] = 1 - hamming(crf_test1.MAP(X_ts[k]), y_ts[k])
print(results.mean())
print(results[results==1].sum() / 100)

0.954
0.62


In [71]:
import sklearn
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [64]:
num_node = 0
for k in range(y_tr.shape[0]):
    for j in range(y_tr[k].shape[0]):
        num_node += 1
        

X_s, y_s = np.zeros((num_node,3)), np.zeros(num_node,dtype='i4')
l = 0
for k in range(y_tr.shape[0]):
    for j in range(y_tr[k].shape[0]):
        X_s[l] = X_tr[k][j,:]
        y_s[l] = y_tr[k][j]
        l+= 1

In [78]:
clf_s = SVC(kernel='linear')
clf_s.fit(X_s,y_s)
clf_l = LogisticRegression()
clf_l.fit(X_s,y_s)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [79]:
ham_svc = np.zeros(y_ts.shape[0])
ham_l = np.zeros(y_ts.shape[0])
for k in range(y_ts.shape[0]):
    ham_svc[k] = accuracy_score(y_ts[k], clf_s.predict(X_ts[k]))
    ham_l[k] = accuracy_score(y_ts[k], clf_l.predict(X_ts[k]))
    
print('SVC Avg Hamming: ', 100*ham_svc.mean(), '%')
print('SVC Zero-One loss: ', 100*ham_svc[ham_svc==1].sum()/y_ts.shape[0])
print('Log Avg Hamming: ', 100*ham_l.mean(), '%')
print('Log Zero-One loss: ', 100*ham_l[ham_l==1].sum()/y_ts.shape[0])

SVC Avg Hamming:  92.3 %
SVC Zero-One loss:  48.0
Log Avg Hamming:  92.3 %
Log Zero-One loss:  49.0


In [69]:
import pandas as pd

hamm = [0.923, 0.923, 0.954]
zo = [0.480, 0.490, 0.620]
models = ['SVC', 'LogReg', 'CRF']
met = ['Average Hamming', 'Zero-One']
res_df = pd.DataFrame(hamm)
res_df.columns = ['Average Hamming:']
res_df['Zero-One:'] = zo
res_df.columns.name = 'Model:'
res_df.index = models
def highlight_max(s):
    is_max = s == s.max()
    return ['color: red' if v else '' for v in is_max]
res_df.style.apply(highlight_max)

In [24]:
# Simultaneuos Pertubation Stochastic Approximation
from scipy.stats import bernoulli

alpha, gamma = 0.602, 0.101
A, c, a = 5, 0.0001, 0.0001

#crf_test3 = CRF(feat_map_toy,100 + 10**3,50, 10, 5)

k = 0

while k < 50:
    
    alpha_k = a / (A + k +1)**alpha
    c_k = c / (k + 1)**gamma
    del_k = 2* c_k * bernoulli.rvs(0.5,size=crf_test3.K) - c_k
    crf_p = CRF(feat_map_toy,100 + 10**3,50, 10, 5, crf_test3.W + del_k)
    crf_m = CRF(feat_map_toy,100 + 10**3,50, 10, 5, crf_test3.W - del_k)
    
    diff = (crf_p.reg_neg_ll(X_tr,y_tr) - crf_m.reg_neg_ll(X_tr,y_tr)) / 2
    crf_test3.W = crf_test3.W - (alpha_k * diff) * (1 / del_k)
    if k  % 10 == 0:
            
        acc_test = np.zeros(30)
            
        for n in range(30):
            acc_test[n] = 1 - hamming(crf_test3.MAP(X_ts[n]), y_ts[n])
            
        print('Iteration: ',k, '. Current Average Test Hamming Accuracy: ',
            100* acc_test.mean(), '%.')
        
        
    k += 1

acc_test = np.zeros(30)
            
for n in range(30):
    acc_test[n] = 1 - hamming(crf_test3.MAP(X_ts[n]), y_ts[n])
            
print('Iteration: ',k, '. Current Average Test Hamming Accuracy: ',
        100* acc_test.mean(), '%.')
        

Iteration:  0 . Current Average Test Hamming Accuracy:  92.4666666667 %.
Iteration:  10 . Current Average Test Hamming Accuracy:  92.2666666667 %.
Iteration:  20 . Current Average Test Hamming Accuracy:  92.4 %.
Iteration:  30 . Current Average Test Hamming Accuracy:  92.4 %.
Iteration:  40 . Current Average Test Hamming Accuracy:  92.4 %.
Iteration:  50 . Current Average Test Hamming Accuracy:  92.4 %.


In [202]:
2*bernoulli.rvs(0.5,size=10) - 1

array([-1,  1,  1, -1,  1, -1, -1, -1, -1,  1])

In [226]:
crf_test3.W

array([ 0.26569467,  0.42955854, -0.32809504, -0.66751197, -0.21377015,
        0.01204941,  0.48927196, -0.12374394, -0.42415367,  0.26093447,
        0.05492973,  0.67446071,  0.62202448,  0.45847976,  0.5953965 ,
       -0.42086594, -0.53643886, -0.22221041, -0.38168191,  0.30985493,
        0.3494285 ,  0.17120865,  0.22322946, -0.68989205, -0.14076703,
        0.63953407, -0.26233943,  0.12768074, -0.61103329, -0.04919524,
       -0.8038806 , -0.01245953,  0.48799304,  0.1842084 , -0.19853262,
       -0.1131701 , -0.71577879, -0.12735687, -0.20848707, -0.03354356,
       -0.60792553,  0.43303068,  0.17809504, -0.27085521, -0.40819822,
       -0.24451328, -0.09458871,  0.39832734, -0.25120413,  0.24378356,
        0.35284519, -0.10262145,  0.06500082, -0.12969668,  0.35326028,
       -0.28181343,  0.51375231,  0.19135779,  0.08392776, -0.20091778,
       -0.23038371,  0.01619796, -0.06379931,  0.0260595 ,  0.07310725,
        0.15199807,  0.57456157,  0.04200584, -0.00782348, -0.11

In [231]:
crf_test3.MAP(X_f[-16])

(2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0)

In [232]:
y_f[-16]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1], dtype=int32)

In [230]:
results

array([ 1.        ,  0.86666667,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  0.93333333,  1.        ,  1.        ,
        1.        ,  0.86666667,  1.        ,  0.93333333,  0.93333333,
        0.8       ,  0.4       ,  1.        ,  1.        ,  1.        ,
        0.93333333,  0.86666667,  1.        ,  1.        ,  0.8       ,
        0.8       ,  0.86666667,  1.        ,  0.8       ,  0.86666667,
        0.93333333,  1.        ,  1.        ,  0.86666667,  1.        ,
        1.        ,  0.93333333,  0.8       ,  1.        ,  1.        ,
        0.93333333,  0.86666667,  1.        ,  1.        ,  0.8       ,
        1.        ,  0.8       ,  1.        ,  0.93333333,  0.46666667,
        0.86666667,  1.        ,  1.        ,  0.93333333,  0.93333333,
        1.        ,  1.        ,  0.93333333,  0.86666667,  1.        ,
        1.        ,  1.        ,  1.        ,  0.73333333,  1.        ,
        0.93333333,  0.93333333,  0.93333333,  1.        ,  1.  